In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import matplotlib.pyplot as plt
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
df=pd.read_csv("spam.csv", encoding="ISO-8859-1")

In [ ]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
df=df[["v1","v2"]]

In [ ]:
le=LabelEncoder()
df["v1"]=le.fit_transform(df["v1"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
Noun_tags=["NN", "NNP", "NNPS", "NNS"]
Verb_tags=["VB","VBD","VBG", "VBN", "VBP", "VBZ"]

In [ ]:
lemmatizer = WordNetLemmatizer()
def prat_lemmatize(token, tag):
  if tag in Noun_tags:
    return lemmatizer.lemmatize(token, "n")
  elif tag in Verb_tags:
    return lemmatizer.lemmatize(token, "v")
  else:
    return lemmatizer.lemmatize(token, "n")

In [ ]:
stop_words = set(stopwords.words('english')) #creating a set of stop words
def cleanResume(resumeText):
  
    resumeText=resumeText.lower()
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)
    resumeText = re.sub(r"what's","what is" ,resumeText)
    resumeText = re.sub(r"\ 've","have " ,resumeText)
    resumeText = re.sub(r"can' t","can not" ,resumeText)
    resumeText = re.sub(r"i'm","i am" ,resumeText)
    resumeText = re.sub(r"\ 're","are" ,resumeText)
    word_tokens = word_tokenize(resumeText) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] #Contains words other than stop words
    tagged_corpus=pos_tag(filtered_sentence)
    tagged_corpus_list=[prat_lemmatize(token, tag) for token, tag in tagged_corpus]
    resumeText=' '.join(tagged_corpus_list)
    return resumeText

In [ ]:
df['v2'] = df.v2.apply(lambda x: cleanResume(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['v1'])

In [ ]:
vectorizer = TfidfVectorizer(min_df=5, ngram_range=[1,3])
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

feature_names = vectorizer.get_feature_names()
idfs = vectorizer.idf_
names_idfs = list(zip(feature_names, idfs))

smallest = sorted(names_idfs, key=operator.itemgetter(1))[:20]
smallest = pd.Series([features[1] for features in smallest], index=[features[0] for features in smallest])

largest = sorted(names_idfs, key=operator.itemgetter(1), reverse=True)[:20]

largest = sorted(largest, key=operator.itemgetter(0))
largest = pd.Series([features[1] for features in largest], index=[features[0] for features in largest])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
largest

06 03                  7.546307
06 03 2nd              7.546307
10am                   7.546307
150pm                  7.546307
1x150p                 7.546307
1x150p wk              7.546307
25                     7.546307
2nd attempt contact    7.546307
2nd attempt reach      7.546307
2nd time               7.546307
2nd time try           7.546307
3030                   7.546307
3030 valid             7.546307
3030 valid 12hrs       7.546307
350 award              7.546307
3510i                  7.546307
3510i colour           7.546307
3510i colour phone     7.546307
3g videophones         7.546307
3g videophones call    7.546307
dtype: float64

In [ ]:
smallest

get     3.155982
call    3.204668
go      3.379642
ur      3.861603
come    3.926420
ok      3.948995
know    4.044762
day     4.054863
lt      4.134060
like    4.150681
time    4.156283
send    4.167583
free    4.190572
want    4.220073
good    4.238200
say     4.262893
text    4.361333
love    4.368253
take    4.389307
see     4.396424
dtype: float64

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB(alpha=0.1)
model.fit(X_train_transformed, y_train)

MultinomialNB(alpha=0.1)

In [ ]:
predictions = model.predict(X_test_transformed)

In [ ]:
roc_auc_score(y_test, predictions)

0.9424901957455477

In [ ]:
model.score(X_train_transformed, y_train)

0.9875568317779373

In [ ]:
model.score(X_test_transformed, y_test)

0.9813352476669059

In [ ]:
spam_message= "SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info"

In [ ]:
le.inverse_transform(model.predict(vectorizer.transform([spam_message])))

array(['spam'], dtype=object)